## Анализ и визуализация данных на языке Python

##  Библиотека pandas

Основные структуры библиотеки - объекты **Series** и **DataFrame**.   
**DataFrame** - табличная структура данных, а **Series** - колонка в этой таблице.    
**DataFrame** можно рассматривать как словарь объектов Series, объединенных одним индексом.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
np.set_printoptions(linewidth=110, legacy='1.25', precision=2)

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
np.__version__
pd.__version__

'2.0.2'

'2.2.2'

### Объект Series

Series - это объект, похожий на одномерный массив: он содержит последовательность данных, которая сопровождается индексными метками для доступа к ним.

Создадим простейший объект Series и выведем его на экран:

In [5]:
series = pd.Series([2, 12, 85, 0, 6])
series

,0
0,2
1,12
2,85
3,0
4,6


Индекс - колонка слева, данные - колонка справа.  
Выгрузить только данные можно через свойство ```.values```, выгрузить только индекс - ```.index```.

In [6]:
series.values

array([ 2, 12, 85,  0,  6])

In [7]:
type(series.values) # знакомый нам массив numpy!

numpy.ndarray

In [8]:
series.index

RangeIndex(start=0, stop=5, step=1)

Можно создать Series с заданным индексом:

In [9]:
series2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
series2

,0
d,4
b,7
a,-5
c,3


In [10]:
series2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

Для доступа к данным можно использовать как метки индекса, так и порядковую позицию элемента:

In [11]:
# эти выражения эквиваленты
print(series2.iloc[2])  # Желательно указать явно, что это индекс для строки
print(series2['a'])

-5
-5


Поддерживаются срезы по порядковому номеру и поиск по набору значений индекса:

In [12]:
print(series2[1:-1])
# Когда мы обращаемся по имени индекса, то последний элемент включается!!!
print(series2['d':'a'])
# Порядок следования имен индексов имеет значение
print(series2[['a', 'd', 'c']])
print(series2[['b', 'a']])

b    7
a   -5
dtype: int64
d    4
b    7
a   -5
dtype: int64
a   -5
d    4
c    3
dtype: int64
b    7
a   -5
dtype: int64


Также поддерживается булев индекс - как на чтение, так и на запись:

In [13]:
series2 > 0

,0
d,True
b,True
a,False
c,True


In [14]:
series2[series2 > 0]

,0
d,4
b,7
c,3


In [15]:
series2[series2 <= 3] = 100_500
series2

,0
d,4
b,7
a,100500
c,100500


Broadcasting также поддерживается:

In [16]:
series2

,0
d,4
b,7
a,100500
c,100500


In [17]:
arr = np.array([10, 20, 30, 40])
series4 = arr + series2
print(series4)

d        14
b        27
a    100530
c    100540
dtype: int64


Математические операции

In [18]:
series2 * 2

,0
d,8
b,14
a,201000
c,201000


In [19]:
np.sqrt(series2)

,0
d,2.000000
b,2.645751
a,317.017350
c,317.017350


Также можно выполнять агрегатные запросы к объекту Series - вычислять среднее, сумму и т.д.

In [20]:
series2.mean()

50252.75

In [21]:
series2.size, series2.count(), series2.sum(), series2.prod()

(4, 4, 201011, 282807000000)

In [22]:
series2.min(), series2.max()

(4, 100500)

In [23]:
series2.mean(), series2.median()

(50252.75, 50253.5)

Функция describe() позволяет сразу получить описательные статистики

In [24]:
series2.describe()

,0
count,4.00000
mean,50252.75000
std,58020.52664
min,4.00000
25%,6.25000
50%,50253.50000
75%,100500.00000
max,100500.00000


Объект Series можно рассматривать как словарь с однотипными данными.  
Более того, для создания объекта Series можно использовать готовый словарь:

In [25]:
print(ord('$'))  # код символа
print(chr(36))  # по коду символа получить сам символ
print(chr(int('1F600', base=16)))
print(ord('😀'))

36
$
😀
128512


In [26]:
ex = {chr(int('20AC', base=16)): 10000,
      chr(int('20BD', base=16)): 1000,
      chr(36): 100}
ex_series = pd.Series(ex)
ex_series

,0
€,10000
₽,1000
$,100


In [27]:
ex['€'], ex_series.loc["€"]

(10000, 10000)

In [28]:
sdata = {
    'First': 35000,
    'Second': 71000,
    'Third': 16000,
    'Fourth': 5000
}
series3 = pd.Series(sdata)
series3

,0
First,35000
Second,71000
Third,16000
Fourth,5000


In [29]:
# Проверка идет по индексу
'First' in series3

True

In [30]:
# Проверка идет по индексу
'Zeroth' in series3

False

In [31]:
bad_data = {'a':1, 'b':2.0, 'c':'Hello World'}
obj_series = pd.Series(bad_data)
print(obj_series)

a              1
b            2.0
c    Hello World
dtype: object


In [32]:
obj_series.mean() ## Error

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [33]:
# obj_series['c'] = len(obj_series['c']) # Вариант изменения значения obj_series с индексом 'c'
# obj_series.sum()

In [34]:
print(obj_series[obj_series > 1]) # с таким series этот фокус уже не пройдет

TypeError: '>' not supported between instances of 'str' and 'int'

In [35]:
obj_series.values

array([1, 2.0, 'Hello World'], dtype=object)

### Выравнивание данных

In [36]:
sdata = {
    'First': 35000,
    'Second': 71000,
    'Third': 16000,
    'Fourth': 5000
}
idx_names = ['Zeroth', 'First', 'Second', 'Third']
# выравнивание по индексу(именам)
series4 = pd.Series(sdata, index=idx_names)
mas = series4.values
print(mas)
print(series4)
print(series4.sum(), np.sum(mas), np.nansum(mas), np.any(np.isnan(mas)))

[   nan 35000. 71000. 16000.]
Zeroth        NaN
First     35000.0
Second    71000.0
Third     16000.0
dtype: float64
122000.0 nan 122000.0 True


`NaN` - отсутствующие данные, "Not a Number" - "не число". По этому признаку можно фильтровать данные:

In [37]:
pd.notna(series4) # Булев массив с результатами сравнения

,0
Zeroth,False
First,True
Second,True
Third,True


In [38]:
pd.isna(series4) # Обратная операция

,0
Zeroth,True
First,False
Second,False
Third,False


In [39]:
series4[series4.notnull()]  # notna() тоже самое, что и notnull()

,0
First,35000.0
Second,71000.0
Third,16000.0


Над объектами Series можно выполнять арифметические операции, но при этом важно помнить про выравнивание данных по индексам:  
если индексы встречаются в обоих объектах, арифметическая операция выполняется как обычно,  
но если индекс не найден - вместо результата операции устанавливается ```NaN```.

In [40]:
series3
series3['Fourth'] = '2344.1'
series3 = series3.astype(np.float64)
series3

,0
First,35000
Second,71000
Third,16000
Fourth,5000


<ipython-input-40-4b8ee7513b4f>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2344.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  series3['Fourth'] = '2344.1'


,0
First,35000.0
Second,71000.0
Third,16000.0
Fourth,2344.1


In [41]:
type(series3['Fourth'])

numpy.float64

In [42]:
series3
series4

,0
First,35000.0
Second,71000.0
Third,16000.0
Fourth,2344.1


,0
Zeroth,NaN
First,35000.0
Second,71000.0
Third,16000.0


In [43]:
print(np.nan + np.nan)
print(2344.1 + np.nan)
series3 + series4

nan
nan


,0
First,70000.0
Fourth,NaN
Second,142000.0
Third,32000.0
Zeroth,NaN


In [44]:
5000 + np.nan, np.nan + np.nan

(nan, nan)

Индекс объекта можно изменять "налету" через свойство ```.index```.

In [45]:
series

,0
0,2
1,12
2,85
3,0
4,6


In [46]:
# Меняем индекс целиком
series.index = ['раз', 'два', 'три', 'четыре', 'пять']; series

,0
раз,2
два,12
три,85
четыре,0
пять,6


In [47]:
series.index[2]

'три'

In [48]:
series.index[2] = 'шесть'  ## Ошибка.

TypeError: Index does not support mutable operations

__ВАЖНО__: У объекта Series есть свойство ```.name```, и это свойство играет важную роль в работе с DataFrame.

In [49]:
series
series3.name = 'Numbers'
series3

,0
раз,2
два,12
три,85
четыре,0
пять,6


,Numbers
First,35000.0
Second,71000.0
Third,16000.0
Fourth,2344.1


## Задание:

__Задача №1.__ Создайте объект Series для хранения оценок по какому-либо предмету, например, по "Линейной Алгебре".  
   Пусть он содержит 10 студентов со случайными оценками от 2 до 5. Фамилии студентов должны быть индексами.

In [52]:
import pandas as pd
import numpy as np

# Список фамилий студентов
students = [
    'Иванов', 'Петров', 'Сидоров', 'Кузнецов', 'Смирнов',
    'Васильев', 'Попов', 'Михайлов', 'Новиков', 'Федоров'
]

# Генерация случайных оценок от 2 до 5
grades = np.random.randint(2, 6, size=10)

# Создание Series
linear_algebra_grades = pd.Series(grades, index=students, name="Линейная Алгебра")

print(linear_algebra_grades)

Иванов      3
Петров      3
Сидоров     3
Кузнецов    2
Смирнов     2
Васильев    5
Попов       5
Михайлов    4
Новиков     4
Федоров     2
Name: Линейная Алгебра, dtype: int64


__Задача №2.__ Выведите средний балл для всех, у кого оценка больше или равна 3.

In [53]:
average_grade = linear_algebra_grades[linear_algebra_grades >= 3].mean()
print(f"Средний балл (≥3): {average_grade:.2f}")

Средний балл (≥3): 3.86


__Задача №3.__ Создайте другой объект Series, для хранения оценок по другому предмету, например по "Математическому Анализу".  
   Посчитайте средний балл для каждого студента по этим двум предметам.

In [54]:
# Генерируем оценки для второго предмета
math_analysis_grades = pd.Series(
    np.random.randint(2, 6, size=10),
    index=students,
    name="Математический Анализ"
)

# Объединяем в DataFrame и считаем среднее по строкам
combined_grades = pd.DataFrame({
    "Линейная Алгебра": linear_algebra_grades,
    "Математический Анализ": math_analysis_grades
})

combined_grades["Средний балл"] = combined_grades.mean(axis=1)

print(combined_grades)

          Линейная Алгебра  Математический Анализ  Средний балл
Иванов                   3                      3           3.0
Петров                   3                      4           3.5
Сидоров                  3                      3           3.0
Кузнецов                 2                      4           3.0
Смирнов                  2                      2           2.0
Васильев                 5                      5           5.0
Попов                    5                      3           4.0
Михайлов                 4                      5           4.5
Новиков                  4                      5           4.5
Федоров                  2                      2           2.0


## Дополнение
можно использовать библиотеку **faker** для создания данных:  

https://faker.readthedocs.io/en/master/

_Установка:_
```
conda install faker
```

In [ ]:
from faker import Faker

In [ ]:
fake = Faker('ru_RU')  # Задаем русский регион
# бывает и такое:  'тов. Блохина Александра Вадимовна'
# и такое 'г-н Киселев Зосима Трифонович'
index_name = [fake.name() for _ in range(5)]
index_name

In [ ]:
fake.first_name(), fake.last_name()

In [ ]:
# Полный(full) профиль
fake.profile()

In [ ]:
# Простой профиль
fake.simple_profile()

In [ ]:
# Создание серии из простого профиля
pd.Series(fake.simple_profile())

Дата может быть преобразована в строковое значение и наоборот.

In [ ]:
res = fake.date_of_birth(); res

In [ ]:
# из datetime.datetime в строку
res.strftime('%d-%B-%Y')

In [ ]:
# из строки в datetime.datetime
datetime.strptime('31-03-2022', '%d-%m-%Y')